In [2]:
# pip install accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xxl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xxl", device_map="auto")


2023-04-26 08:22:02.881003: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-26 08:22:02.947923: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 08:22:04.078905: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
import torch

# Evaluation du modèle

In [6]:
from datasets import load_dataset, load_metric

In [1]:
import pandas as pd
pd.set_option('mode.chained_assignment', None) #désactive les warnings 

import os
import s3fs

## Charger les données

In [8]:
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "mberthe/narval"
fs.ls(BUCKET)
BUCKET = "mberthe/narval"
FILE_KEY_S3 = "camemBERT/answers(2).json"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    f1=json.load(file_in)

In [9]:
import collections

In [23]:
import numpy as np # linear algebra
import json # to read json
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

In [11]:
df=pd.DataFrame(f1["data"])
df.head()

,paragraphs
0,[{'qas': [{'question': 'quelle est la valeur ...
1,[{'qas': [{'question': 'quelle est la valeur d...
2,[{'qas': [{'question': 'Quelle est la valeur d...
3,[{'qas': [{'question': 'quelle est la valeur d...
4,[{'qas': [{'question': 'Quelle est la valeur d...


In [12]:
input_file_path = FILE_PATH_S3
record_path = ['data','paragraphs','qas','answers']
rpqs = pd.json_normalize(f1, record_path =record_path,meta=[
        ['data', 'paragraphs','context'], 
        ['data', 'paragraphs', 'qas','id'],
        ['data', 'paragraphs', 'qas','question']])

In [13]:
rpqs=rpqs.rename(columns={"data.paragraphs.context":"context","data.paragraphs.qas.id":"id","data.paragraphs.qas.question":"question"})

In [14]:
rpqs

,answer_id,document_id,question_id,text,answer_start,answer_end,answer_category,context,id,question
0,726620,1364517,665706,"99,8",975,979,None,Rapport sur le prix et la qualité de l’eau pot...,665706,quelle est la valeur du taux de conformité de...
1,726622,1364517,665707,100 %,1009,1014,None,Rapport sur le prix et la qualité de l’eau pot...,665707,quelle est la valeur du taux de conformité des...
2,726502,1364517,665606,"99,8 %",974,982,None,Rapport sur le prix et la qualité de l’eau pot...,665606,Quelle est la valeur de P101.1?
3,726504,1364517,665608,100 %,1008,1014,None,Rapport sur le prix et la qualité de l’eau pot...,665608,Quelle est la valeur de P101.2 ?
4,726505,1364517,665609,107,4037,4041,None,Rapport sur le prix et la qualité de l’eau pot...,665609,Quelle est la valeur de P103.2B
...,...,...,...,...,...,...,...,...,...,...
121,727136,1364611,666176,"2,11",1695,1699,None,\n \n Les indicateurs de performances \n \nD...,666176,Quel est le taux d'impayés sur les factures d'...
122,727138,1364611,666178,"0,20",1778,1783,None,\n \n Les indicateurs de performances \n \nD...,666178,Quel est le taux de réclamation ?
123,727139,1364611,666179,"0,20",1778,1782,None,\n \n Les indicateurs de performances \n \nD...,666179,Quelle est la valeur de P258.1?
124,727150,1364619,666190,"0,014",827,833,None,19 5 Actions de solidarité et de coopération \...,666190,Quelle est la valeur de P109.0?


## Traiter les données: 
- De la forme : Contexte: "..." + Question : "..."
- On supprime les \n

In [17]:
# making copy of question column
new = rpqs["question"].copy()
# concatenating team with name column
# overwriting name column
rpqs["context"]= rpqs["context"].str.cat(new, sep =". Question: ")
rpqs['context']= "Contexte: " + rpqs['context']
# display
#rpqs['context'][0]


In [18]:
def n(text):
        regex="\n"
        return re.sub(regex, " ", text)

In [19]:
import re
for i in rpqs.index:
    rpqs['context'][i]=n(rpqs['context'][i])

## Prédictions

In [21]:
answer_pred=[]
for i in rpqs.index:
    print(i)
    input_text = rpqs['context'][i]
    input_ids = tokenizer(input_text, max_length=510,
        truncation="only_second",
        padding="max_length",return_tensors="pt").input_ids

    outputs = model.generate(input_ids)
    answer_pred.append(tokenizer.decode(outputs[0]))

0


/opt/mamba/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125


In [24]:
answer_pred

['<pad> • 99,8 %</s>',
 '<pad> 100 %</s>',
 '<pad> 0</s>',
 '<pad> 0</s>',
 '<pad> 107 points sur 120</s>',
 '<pad> 79,9 %</s>',
 '<pad> 120 points</s>',
 '<pad> 80%</s>',
 '<pad> 21 947</s>',
 '<pad> 7 312 tonnes de matières sèches</s>',
 '<pad>Quantite de boues d’épuration produites e',
 '<pad> 0,02</s>',
 '<pad> 0</s>',
 '<pad>Il s’agit de détecter d’éventuelles zones à problèmes en',
 '<pad> 43 points noirs</s>',
 '<pad> 100 %</s>',
 '<pad> 100 %</s>',
 '<pad> 1,0 1,3 1,2 1,1 1,5</s>',
 '<pad> 0,7 %</s>',
 '<pad> 73,18%</s>',
 '<pad> 11 203</s>',
 '<pad> 9,96 m3/km/j</s>',
 '<pad> 9,96 m3/km/j</s>',
 '<pad> 9,28 m3/km/j</s>',
 '<pad> 0,02</s>',
 '<pad> 0 Calcul</s>',
 '<pad> 95,52 %</s>',
 '<pad> 1 jours</s>',
 '<pad> 1 jours</s>',
 '<pad> 0,96 %</s>',
 '<pad> 0 %</s>',
 '<pad> 0,96 %</s>',
 '<pad> 0 %</s>',
 '<pad> 115</s>',
 '<pad> m3/km/j</s>',
 '<pad> 115</s>',
 '<pad> €/m3</s>',
 '<pad> €/m3</s>',
 '<pad> 78 0 100 %</s>',
 '<pad> 78 0 100 %</s>',
 '<pad> 80 %</s>',
 '<pad> 2.1

In [25]:
rpqs['answer_pred']=answer_pred
rpqs[['text','answer_pred']].sample(15)

,text,answer_pred
50,"e 0,36%","<pad> 0,36%</s>"
29,"0,96","<pad> 0,96 %</s>"
101,213 372,<pad> 213 372</s>
97,"82,24","<pad> 82,24 %</s>"
71,"4,22","<pad> 3,45 4,05 3,90 4,09 4,22</s>"
53,"8,9","<pad> 8,9 m3 / km / jour</s>"
115,"94,3 %","<pad> 94,3 %</s>"
14,43,<pad> 43 points noirs</s>
17,"1,5","<pad> 1,0 1,3 1,2 1,1 1,5</s>"
36,"0,0010 €",<pad> €/m3</s>


## Normalisation et évaluation des prédictions

In [27]:
def normalize_text(s):
    """removing caractères, /, accents, m3, /n"""
    import string, re
    
    def no_unit(text):
        regex="m3"
        return re.sub(regex, " ", text)
    def no_exp(text):
        regex="m³"
        return re.sub(regex, " ", text)
    def no_punct(text):
        regex="\\.(?!\d)"
        return re.sub(regex," ",text)
   
    def white_space_fix(text):
        return " ".join(text.split())
    def no_date(text):
        date_extract_pattern = "[0-9]{1,2}\\/[0-9]{1,2}\\/[0-9]{4}"
        return re.sub(date_extract_pattern,"",text)
    def special_char(text):
        regex=regex = "[^\w\s,.]"
        return re.sub(regex, " ", text)
   #on retire tout les caractère des réponses 
    def no_char(text):
        regex=r"([a-zA-Z])"
        return re.sub(regex, " ", text)
    def comma(text):
        regex=",(?!\d)"
        return re.sub(regex," ",text)
    def indic(text):
        regex="(P\d\d\d\\.\d\s*)|(D\d\d\d\\.\d\s*)|(VP\\.\d\d\d)"
        return re.sub(regex," ", text)

    return white_space_fix(no_punct(no_char(unidecode(comma(special_char(indic(no_exp(no_unit(no_date(s))))))))))

### Exact Match

In [28]:
def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

In [29]:
from unidecode import unidecode
import statistics

In [30]:
em_list1=[]
for i in rpqs.index:
    truth=rpqs["text"][i]
    prediction1=rpqs["answer_pred"][i] 
    em1=compute_exact_match(prediction1,truth)
    em_list1.append(em1)
moyenne_em1=statistics.mean(em_list1)
#print('avant le traitement des predictions: em_moyen = 0,10')
print(' em_moyen=',moyenne_em1)

 em_moyen= 0.6587301587301587


### F1

In [31]:
def compute_f1(prediction, truth):
    pred_tokens =normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [32]:
f1_list=[]
for i in rpqs.index:
    truth=rpqs["text"][i]
    prediction=rpqs["answer_pred"][i] 
    f1=compute_f1(prediction,truth)
    #print(f1)
    f1_list.append(f1)
#print(f1_list)
moyenne_f1=statistics.mean(f1_list)
print('f1_moyen après traitement=',moyenne_f1)


f1_moyen après traitement= 0.7123015873015873


### Accuracy

In [33]:
def compute_acc(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    if len(common_tokens) == 0:
        return 0
    else :
        return 1

In [34]:
acc_list=[]
for i in rpqs.index:
    truth=rpqs["text"][i]
    #print(i,truth)
    prediction=rpqs["answer_pred"][i] 
    #print(i,prediction)
    acc=compute_acc(prediction,truth)
    #print("acc",acc)
    acc_list.append(acc)
#print(f1_list)
moyenne_acc=statistics.mean(acc_list)
print('acc_moyen après traitement=',moyenne_acc)

acc_moyen après traitement= 0.7698412698412699


In [35]:
for i in rpqs.index:
    rpqs['answer_pred'][i]=normalize_text(rpqs['answer_pred'][i])
    rpqs['text'][i]=normalize_text(rpqs['text'][i])

In [36]:
rpqs[['text','answer_pred']].sample(15)

,text,answer_pred
122,"0,20","0,20 1000"
25,0,0
68,"10,6","10,6"
75,"2,22","2,22"
42,"2,22","2,22 365 378"
101,213 372,213 372
4,107,107 120
109,100,100
90,"0 ,92","0,92"
49,"0,36","0,36"


## Enregistrer les résultats

In [37]:
BUCKET_OUT = "mberthe/narval"
FILE_KEY_OUT_S3 = "camemBERT/pred_flan-xxl.csv"
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w', encoding="utf-8") as file_out:
    rpqs.to_csv(file_out)